In [30]:
#步骤一 数据加载与预处理
#1:读取数据表
#2:修改字段‘report_date’的日期类型
#3:添加时间维度，显示每一个‘report date’所在的年、月、日、周、周历天

import pandas as pd

data = pd.read_csv('./user_balance_table.csv')

def add_timestamp(data):
    data['report_date'] = pd.to_datetime(data['report_date'], format='%Y%m%d')
    data['day'] = data['report_date'].dt.day
    data['month'] = data['report_date'].dt.month
    data['year'] = data['report_date'].dt.year
    data['week'] = data['report_date'].dt.week
    data['weekday'] = data['report_date'].dt.weekday
    return data
data = add_timestamp(data)
data


,user_id,report_date,tBalance,yBalance,total_purchase_amt,direct_purchase_amt,purchase_bal_amt,purchase_bank_amt,total_redeem_amt,consume_amt,...,share_amt,category1,category2,category3,category4,day,month,year,week,weekday
0,1,2014-08-05,20385,20383,2,0,0,0,0,0,...,2,NaN,NaN,NaN,NaN,5,8,2014,32,1
1,1,2014-08-08,20391,20389,2,0,0,0,0,0,...,2,NaN,NaN,NaN,NaN,8,8,2014,32,4
2,1,2014-08-11,20397,20395,2,0,0,0,0,0,...,2,NaN,NaN,NaN,NaN,11,8,2014,33,0
3,1,2014-08-14,20403,20401,2,0,0,0,0,0,...,2,NaN,NaN,NaN,NaN,14,8,2014,33,3
4,1,2014-08-17,20409,20407,2,0,0,0,0,0,...,2,NaN,NaN,NaN,NaN,17,8,2014,33,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2840416,28033,2014-08-25,550646,550585,61,0,0,0,0,0,...,61,NaN,NaN,NaN,NaN,25,8,2014,35,0
2840417,28033,2014-08-31,525707,538147,60,0,0,0,12500,12500,...,60,0.0,0.0,0.0,12500.0,31,8,2014,35,6
2840418,28033,2014-07-24,20487121,20484824,2297,0,0,0,0,0,...,2297,NaN,NaN,NaN,NaN,24,7,2014,30,3
2840419,28033,2014-07-27,20462288,20491722,2298,0,0,0,31732,0,...,2298,NaN,NaN,NaN,NaN,27,7,2014,30,6


In [31]:
#步骤二 数据备份并选取需要的数据字作为训练数据集
#1：数据备份
#2：选取2014-03-01之后的数据字段‘total_purchase_amt’‘total_redeem_amt’

def get_total_balance(data, begin):
    df_temp = data.copy()
    df_temp = df_temp.groupby(['report_date'])['total_purchase_amt','total_redeem_amt'].sum()
    df_temp.reset_index(inplace=True)
    df_temp = df_temp[(df_temp['report_date']>=begin)]
    return df_temp     
total_balance = get_total_balance(data,'2014-03-01')
total_balance

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


,report_date,total_purchase_amt,total_redeem_amt
243,2014-03-01,362865580,211279011
244,2014-03-02,276202230,246199417
245,2014-03-03,505305862,513017360
246,2014-03-04,524146340,250562978
247,2014-03-05,454295491,209072753
...,...,...,...
422,2014-08-27,302194801,468164147
423,2014-08-28,245082751,297893861
424,2014-08-29,267554713,273756380
425,2014-08-30,199708772,196374134


In [32]:
#步骤三 生成测试数据集
#1：生成待预测时间框架2014-09-01~2014-09-30
#2：将训练数据集合待预测时间框架数据集合并同一张表待用
#3：在新合并工作表中添加时间时间维度，显示每一个‘report date’所在的年、月、日、周、周历天

import numpy as np
import datetime

def generate_test_data(data):
    total_balance = data.copy()
    start = datetime.datetime(2014,9,1)
    end = datetime.datetime(2014,10,1)
    testdata = []
    while start != end:
        temp = [start, np.nan, np.nan]
        testdata.append(temp)
        start = start + datetime.timedelta(days=1)
    testdata = pd.DataFrame(testdata)
    testdata.columns = total_balance.columns
    result = pd.concat([total_balance,testdata],axis=0)
    return result

total_balance = generate_test_data(total_balance)
total_balance = add_timestamp(total_balance)
total_balance

,report_date,total_purchase_amt,total_redeem_amt,day,month,year,week,weekday
243,2014-03-01,362865580.0,211279011.0,1,3,2014,9,5
244,2014-03-02,276202230.0,246199417.0,2,3,2014,9,6
245,2014-03-03,505305862.0,513017360.0,3,3,2014,10,0
246,2014-03-04,524146340.0,250562978.0,4,3,2014,10,1
247,2014-03-05,454295491.0,209072753.0,5,3,2014,10,2
...,...,...,...,...,...,...,...,...
25,2014-09-26,NaN,NaN,26,9,2014,39,4
26,2014-09-27,NaN,NaN,27,9,2014,39,5
27,2014-09-28,NaN,NaN,28,9,2014,39,6
28,2014-09-29,NaN,NaN,29,9,2014,40,0


In [33]:
#步骤三 生成测试数据集
#4: 在测试数据集中使用中文日历标注出周历天和节假日

import datetime
from chinese_calendar import is_workday, is_holiday

total_balance['is_holiday'] = total_balance['report_date'].apply(lambda x: is_holiday(x))
total_balance['is_holiday'] = total_balance['is_holiday']. replace({True:1, False:0})
total_balance

,report_date,total_purchase_amt,total_redeem_amt,day,month,year,week,weekday,is_holiday
243,2014-03-01,362865580.0,211279011.0,1,3,2014,9,5,1
244,2014-03-02,276202230.0,246199417.0,2,3,2014,9,6,1
245,2014-03-03,505305862.0,513017360.0,3,3,2014,10,0,0
246,2014-03-04,524146340.0,250562978.0,4,3,2014,10,1,0
247,2014-03-05,454295491.0,209072753.0,5,3,2014,10,2,0
...,...,...,...,...,...,...,...,...,...
25,2014-09-26,NaN,NaN,26,9,2014,39,4,0
26,2014-09-27,NaN,NaN,27,9,2014,39,5,1
27,2014-09-28,NaN,NaN,28,9,2014,39,6,0
28,2014-09-29,NaN,NaN,29,9,2014,40,0,0


In [34]:
#步骤三 生成测试数据集
#5: 根据实际工作休假日期修正日历判别的工作日和节假日，即：将工作的周六日设置为周一，将休假的工作日设置为周日

re_weekday = []
for index, (weekday, is_holiday) in enumerate(zip(total_balance['weekday'].values,total_balance['is_holiday'].values)):
    r = total_balance['weekday'].values[index]
    if weekday not in [5,6] and is_holiday == 1:
        r = 6
    elif weekday in [5,6] and is_holiday ==0:
        r = 0
    re_weekday.append(r)
total_balance['weekday'] = re_weekday
        

In [35]:
#步骤四 创建新的工作表完成后续步骤
#1：创建新的工作表temp

temp = total_balance.copy()

In [44]:
#步骤五 计算周期因子
#1: 计算每个周历天的平均值，展示在3个表字段中'weekday','purchase_weekday','redeem_weekday'
#2：计算purchase和redeem的周期因子

total_balance = temp[temp['report_date']<'2014-09-01'].copy()
weekday_weight = total_balance[['weekday','total_purchase_amt','total_redeem_amt']].groupby('weekday',as_index=False).mean()
weekday_weight.columns = ['weekday','purchase_weekday','redeem_weekday']
weekday_weight['purchase_weekday']/=np.mean(total_balance['total_purchase_amt'])
weekday_weight['redeem_weekday']/=np.mean(total_balance['total_redeem_amt'])
weekday_weight

,weekday,purchase_weekday,redeem_weekday
0,0,1.225286,1.317867
1,1,1.209608,1.172248
2,2,1.163752,1.157944
3,3,1.157122,1.046964
4,4,0.920635,0.996817
5,5,0.709237,0.669791
6,6,0.685572,0.709648


In [45]:
#步骤六 从周期因子转换为日期因子

total_balance = pd.merge(total_balance,weekday_weight,on='weekday',how='left')
weekday_count = total_balance[['report_date','day','weekday']].groupby(['day','weekday'],as_index=False).count()
weekday_count = pd.merge(weekday_count,weekday_weight,on='weekday')
weekday_count['purchase_weekday'] = weekday_count['purchase_weekday']*weekday_count['report_date']/len(np.unique(total_balance['month']))
weekday_count['redeem_weekday'] = weekday_count['redeem_weekday']*weekday_count['report_date']/len(np.unique(total_balance['month']))
day_rate = weekday_count.drop(['weekday','report_date'],axis=1).groupby('day',as_index=False).sum()
day_rate

,day,purchase_weekday,redeem_weekday
0,1,0.903372,0.905067
1,2,0.848910,0.852437
2,3,1.023991,0.993914
3,4,1.110867,1.159927
4,5,1.029040,1.005767
5,6,0.970377,0.965545
6,7,0.976934,0.983172
7,8,0.981964,0.961286
8,9,0.978039,1.001669
9,10,1.023991,0.993914


In [46]:
#步骤七 使用日期因子完成周期预测
#1: 计算日历天的均值

day_mean = total_balance[['day','total_purchase_amt','total_redeem_amt']].groupby('day',as_index=False).mean()
day_mean

,day,total_purchase_amt,total_redeem_amt
0,1,3.253391e+08,2.347676e+08
1,2,2.481255e+08,2.186074e+08
2,3,2.994808e+08,2.968247e+08
3,4,3.159802e+08,2.925598e+08
4,5,3.286272e+08,2.552484e+08
5,6,2.998453e+08,2.460958e+08
6,7,2.836725e+08,2.374837e+08
7,8,2.918664e+08,2.475255e+08
8,9,2.856603e+08,2.440269e+08
9,10,3.446347e+08,2.593854e+08


In [47]:
#步骤七 使用日期因子完成周期预测
#2: 使用日历天均值和大盘值得到日期因子

day_base = pd.merge(day_mean, day_rate, on='day', how='left')
day_base['total_purchase_amt'] /= day_base['purchase_weekday']
day_base['total_redeem_amt'] /= day_base['redeem_weekday']
day_base

,day,total_purchase_amt,total_redeem_amt,purchase_weekday,redeem_weekday
0,1,3.601385e+08,2.593926e+08,0.903372,0.905067
1,2,2.922873e+08,2.564498e+08,0.848910,0.852437
2,3,2.924642e+08,2.986424e+08,1.023991,0.993914
3,4,2.844447e+08,2.522226e+08,1.110867,1.159927
4,5,3.193531e+08,2.537847e+08,1.029040,1.005767
5,6,3.089988e+08,2.548776e+08,0.970377,0.965545
6,7,2.903702e+08,2.415485e+08,0.976934,0.983172
7,8,2.972272e+08,2.574941e+08,0.981964,0.961286
8,9,2.920746e+08,2.436204e+08,0.978039,1.001669
9,10,3.365602e+08,2.609738e+08,1.023991,0.993914


In [48]:
#步骤七 使用日期因子完成周期预测
#3: 在表中添加预测时间字段report_date

for index, row in day_base.iterrows():
    if row['day'] == 31:
       break
    day_base.loc[index,'report_date'] = pd.to_datetime('2014/09/'+str(int(row['day'])))
day_base

,day,total_purchase_amt,total_redeem_amt,purchase_weekday,redeem_weekday,report_date
0,1,3.601385e+08,2.593926e+08,0.903372,0.905067,2014-09-01
1,2,2.922873e+08,2.564498e+08,0.848910,0.852437,2014-09-02
2,3,2.924642e+08,2.986424e+08,1.023991,0.993914,2014-09-03
3,4,2.844447e+08,2.522226e+08,1.110867,1.159927,2014-09-04
4,5,3.193531e+08,2.537847e+08,1.029040,1.005767,2014-09-05
5,6,3.089988e+08,2.548776e+08,0.970377,0.965545,2014-09-06
6,7,2.903702e+08,2.415485e+08,0.976934,0.983172,2014-09-07
7,8,2.972272e+08,2.574941e+08,0.981964,0.961286,2014-09-08
8,9,2.920746e+08,2.436204e+08,0.978039,1.001669,2014-09-09
9,10,3.365602e+08,2.609738e+08,1.023991,0.993914,2014-09-10


In [49]:
#步骤七 使用日期因子完成周期预测
#4: 使用日期因子和大盘数值完成预测

day_base['weekday'] = day_base['report_date'].dt.weekday
day_base

,day,total_purchase_amt,total_redeem_amt,purchase_weekday,redeem_weekday,report_date,weekday
0,1,3.601385e+08,2.593926e+08,0.903372,0.905067,2014-09-01,0.0
1,2,2.922873e+08,2.564498e+08,0.848910,0.852437,2014-09-02,1.0
2,3,2.924642e+08,2.986424e+08,1.023991,0.993914,2014-09-03,2.0
3,4,2.844447e+08,2.522226e+08,1.110867,1.159927,2014-09-04,3.0
4,5,3.193531e+08,2.537847e+08,1.029040,1.005767,2014-09-05,4.0
5,6,3.089988e+08,2.548776e+08,0.970377,0.965545,2014-09-06,5.0
6,7,2.903702e+08,2.415485e+08,0.976934,0.983172,2014-09-07,6.0
7,8,2.972272e+08,2.574941e+08,0.981964,0.961286,2014-09-08,0.0
8,9,2.920746e+08,2.436204e+08,0.978039,1.001669,2014-09-09,1.0
9,10,3.365602e+08,2.609738e+08,1.023991,0.993914,2014-09-10,2.0


In [52]:
#步骤八 输出预测结果

day_pred = day_base[['day','total_purchase_amt','total_redeem_amt','report_date','weekday']]
day_pred = pd.merge(day_pred,weekday_weight,on='weekday')
day_pred['total_purchase_amt']*=day_pred['purchase_weekday']
day_pred['total_redeem_amt']*=day_pred['redeem_weekday']
day_pred = day_pred.sort_values('report_date')[['report_date','total_purchase_amt','total_redeem_amt']]
day_pred['report_date']=day_pred['report_date'].apply(lambda x: str(x).replace('-','')[0:8])
day_pred

,report_date,total_purchase_amt,total_redeem_amt
0,20140901,4.412725e+08,3.418450e+08
5,20140902,3.535532e+08,3.006227e+08
10,20140903,3.403557e+08,3.458112e+08
14,20140904,3.291372e+08,2.640680e+08
18,20140905,2.940077e+08,2.529769e+08
22,20140906,2.191534e+08,1.707146e+08
26,20140907,1.990697e+08,1.714145e+08
1,20140908,3.641882e+08,3.393430e+08
6,20140909,3.532959e+08,2.855835e+08
11,20140910,3.916725e+08,3.021931e+08


In [53]:
day_pred.to_csv('rule_base_chinese_130+.csv',index=False,header=None)